<a href="https://colab.research.google.com/github/abdala9512/nlp-evaluacion-problemas-sociales/blob/main/src/notebooks/Skip_Grams_OmarBalcero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tensorflow as tf
import nltk
import numpy as np
import gensim
from gensim.models import Word2Vec
import multiprocessing
from sklearn.manifold import TSNE
from bokeh.io import output_notebook
from bokeh.plotting import show, figure
%matplotlib inline
import keras
from sklearn.metrics import roc_curve, auc, roc_auc_score, classification_report, confusion_matrix, make_scorer

In [ ]:
from sklearn.svm import SVC
import statistics
from sklearn.model_selection import train_test_split

In [ ]:
print(gensim.__version__)

4.2.0


In [ ]:
import numpy as np
from keras.layers import LSTM
from keras.layers.core import Dense
from keras_preprocessing.sequence import pad_sequences

In [ ]:
!pip install --upgrade gensim 

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
df=pd.read_csv("ArchivoProblemas (ajustado).csv",sep=";")
df=df.drop("Unnamed: 4", axis=1)

In [ ]:
df.head()

,TIPO,ID,DESCRIPCION,DESCRIPCION_LEM
0,Problem,3655.0,"Deficiencia en las reglas, protocolos y proces...","['deficiencia', 'en', 'el', 'regla', ',', 'pro..."
1,Effect,18181.0,Descoordinacion institucional en el tema de at...,"['descoordinacion', 'institucional', 'en', 'el..."
2,Effect,18182.0,Ineficiencia e ineficacia en la atencion a vic...,"['ineficiencia', 'e', 'ineficacia', 'en', 'el'..."
3,Effect,18183.0,Inequidad social,"['inequidad', 'social']"
4,Effect,18626.0,Baja capacidad de las entidades territoriales ...,"['bajo', 'capacidad', 'de', 'el', 'entidad', '..."


In [ ]:
df['DESC_LEM']=df.apply(lambda x:str(x['DESCRIPCION_LEM']).replace(",","").replace("'","").replace("[","").replace("]","").split() ,axis=1)

In [ ]:
w2v=Word2Vec(df['DESC_LEM'],sg=1, vector_size=5,window=5,min_count=5,negative=15,workers=multiprocessing.cpu_count(),seed=123)

In [ ]:
word_vectors =w2v.wv
result= word_vectors.similar_by_word("social")
print("Parecidas a inequidad", result[:5])

Parecidas a inequidad [('asi', 0.9924023151397705), ('acción', 0.9853375554084778), ('mayor', 0.9838107228279114), ('sitio', 0.9835538268089294), ('oportunidad', 0.9828771948814392)]


In [ ]:
X=w2v.wv[w2v.wv.index_to_key]

In [ ]:
word_vectors['social']

array([-0.08456306,  0.15048912,  0.89405984,  0.7126953 ,  1.3297857 ],
      dtype=float32)

In [ ]:
tsne=TSNE(n_components=2, n_iter=250)

In [ ]:
X2d=tsne.fit_transform(X)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


In [ ]:
wv_df=pd.DataFrame(X2d, columns=['x','y'])
wv_df['token']=w2v.wv.key_to_index

In [ ]:
wv_df.head()

,x,y,token
0,-0.488374,-0.122019,de
1,-0.687990,-0.399679,el
2,-0.667889,-0.317175,en
3,-0.329927,0.172402,y
4,-0.362082,0.095296,.


VISUALIZACION

In [ ]:
output_notebook()

In [ ]:
p= figure(plot_width =800, plot_height=800)
_=p.text(x=wv_df.x, y=wv_df.y, text= wv_df.token)

In [ ]:
show(p)

# NO STOP_WORDS

In [ ]:
stop_words = nltk.corpus.stopwords.words('spanish')

In [ ]:
df['DESC_LEM2']=df.apply(lambda x:[ i for i in x['DESC_LEM'] if i not in stop_words] , axis=1 )

In [ ]:
df.head()

,TIPO,ID,DESCRIPCION,DESCRIPCION_LEM,DESC_LEM,DESC_LEM2
0,Problem,3655.0,"Deficiencia en las reglas, protocolos y proces...","['deficiencia', 'en', 'el', 'regla', ',', 'pro...","[deficiencia, en, el, regla, protocolo, y, pro...","[deficiencia, regla, protocolo, proceso, corre..."
1,Effect,18181.0,Descoordinacion institucional en el tema de at...,"['descoordinacion', 'institucional', 'en', 'el...","[descoordinacion, institucional, en, el, tema,...","[descoordinacion, institucional, tema, atencio..."
2,Effect,18182.0,Ineficiencia e ineficacia en la atencion a vic...,"['ineficiencia', 'e', 'ineficacia', 'en', 'el'...","[ineficiencia, e, ineficacia, en, el, atencion...","[ineficiencia, ineficacia, atencion, victima, ..."
3,Effect,18183.0,Inequidad social,"['inequidad', 'social']","[inequidad, social]","[inequidad, social]"
4,Effect,18626.0,Baja capacidad de las entidades territoriales ...,"['bajo', 'capacidad', 'de', 'el', 'entidad', '...","[bajo, capacidad, de, el, entidad, territorial...","[bajo, capacidad, entidad, territorial, implem..."


In [ ]:
w2v=Word2Vec(df['DESC_LEM2'],sg=1, vector_size=10,window=5,min_count=5,negative=15,workers=multiprocessing.cpu_count(),seed=123)

In [ ]:
word_vectors =w2v.wv
result= word_vectors.similar_by_word("impacto")
print("Parecidas a inequidad", result[:5])

Parecidas a inequidad [('solicitud', 0.9860308170318604), ('desconocimiento', 0.9846330285072327), ('potencial', 0.9844119548797607), ('asi', 0.9833272695541382), ('orientado', 0.982312023639679)]


In [ ]:
X=w2v.wv[w2v.wv.index_to_key]

In [ ]:
tsne=TSNE(n_components=2, n_iter=250)
X2d=tsne.fit_transform(X)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


In [ ]:
wv_df=pd.DataFrame(X2d, columns=['x','y'])
wv_df['token']=w2v.wv.key_to_index
wv_df.head()

,x,y,token
0,-0.038962,-0.009852,.
1,0.669983,0.194527,via
2,0.974067,0.278530,municipio
3,-0.125016,-0.037210,aumento
4,-0.144800,-0.042485,bajo


VISUALIZACIÓN

In [ ]:
output_notebook()
p= figure(plot_width =800, plot_height=800)
_=p.text(x=wv_df.x, y=wv_df.y, text= wv_df.token)

In [ ]:
show(p)

## MODELO 

In [ ]:
words = set(w2v.wv.index_to_key )
df['Text_vect'] = np.array([np.array([w2v.wv[i] for i in ls if i in words])
                         for ls in df['DESC_LEM2']])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
df.head(2)

,TIPO,ID,DESCRIPCION,DESCRIPCION_LEM,DESC_LEM,DESC_LEM2,Text_vect,Text_vect_avg
0,Problem,3655.0,"Deficiencia en las reglas, protocolos y proces...","['deficiencia', 'en', 'el', 'regla', ',', 'pro...","[deficiencia, en, el, regla, protocolo, y, pro...","[deficiencia, regla, protocolo, proceso, corre...","[[-0.53367513, -0.87996334, -0.8490511, -0.473...","[-0.22759782, -0.08495624, 0.14136416, -0.1369..."
1,Effect,18181.0,Descoordinacion institucional en el tema de at...,"['descoordinacion', 'institucional', 'en', 'el...","[descoordinacion, institucional, en, el, tema,...","[descoordinacion, institucional, tema, atencio...","[[0.027042698, -0.5320292, -0.5438745, -0.9339...","[-0.22100642, -0.0865075, 0.15473725, -0.23886..."


In [ ]:
content_sentence1_Text_vect = list(df['Text_vect'].loc[0:0])
content_sentence1_Text_vect

[array([[-0.53367513, -0.87996334, -0.8490511 , -0.4739273 ,  0.77375144,
         -0.34449047, -1.2473748 ,  0.41000578, -0.04962339,  0.4808008 ],
        [ 0.10674752,  0.03309609, -0.731666  , -0.625797  ,  0.51830256,
         -0.48064154, -1.0466671 ,  0.1556246 ,  0.11837637,  0.17286602],
        [ 0.16184433, -0.55664444, -0.43475133, -0.99536324,  0.53490275,
         -0.75435525, -1.1851883 , -0.165634  , -0.22603962,  0.41481948],
        [ 0.6060476 , -0.25873792, -0.21389972, -1.0862544 ,  0.93726444,
         -1.1630435 , -0.98558235, -0.01601527,  0.59181595, -0.1396401 ],
        [ 0.17542076, -0.16822302, -0.70778733, -0.5599721 ,  0.3736908 ,
         -0.29938102, -1.2572054 ,  0.05184021, -0.0361829 ,  0.5030252 ],
        [ 0.58512104, -0.4639244 , -0.29078364, -1.1404446 ,  0.1942224 ,
         -0.7642843 , -1.325434  ,  0.14151125, -0.23046818,  0.5380755 ],
        [ 0.02088889,  0.14159341, -0.68744016, -0.65269715,  0.4181891 ,
         -0.51891744, -1.0947098

In [ ]:
df['DESC_LEM2'][0]

['deficiencia',
 'regla',
 'protocolo',
 'proceso',
 'corresponsabilidad',
 'nacion',
 'dema',
 'ente',
 'territorial',
 'pais',
 'implementacion',
 'politica',
 'victima',
 'dema',
 'actor',
 'conflicto']

In [ ]:
text_vect_avg = []
for v in df['Text_vect']:
    if v.size:
        text_vect_avg.append(v.mean(axis=0))
    else:
        text_vect_avg.append(np.zeros(10, dtype=float)) 
        
        
df['Text_vect_avg'] = text_vect_avg


In [ ]:
df["TIPO"].value_counts()

Effect     5976
Cause      5307
Problem    1751
Name: TIPO, dtype: int64

In [ ]:
df_Machine_Learning = pd.DataFrame(text_vect_avg)
df_Machine_Learning.head(2)

,0,1,2,3,4,5,6,7,8,9
0,0.286256,-0.339174,-0.573575,-0.884887,0.445071,-0.710361,-1.126215,0.090227,0.164009,0.232113
1,0.226526,-0.461072,-0.451982,-1.173649,0.149041,-0.719426,-1.211360,0.219315,0.295546,0.043293


In [ ]:
df_Machine_Learning.columns = ['Feature_' + str(i+1) for i in range(0, df_Machine_Learning.shape[1])]
df_Machine_Learning.head(2)

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10
0,0.286256,-0.339174,-0.573575,-0.884887,0.445071,-0.710361,-1.126215,0.090227,0.164009,0.232113
1,0.226526,-0.461072,-0.451982,-1.173649,0.149041,-0.719426,-1.211360,0.219315,0.295546,0.043293


In [ ]:
final_df = pd.concat([df['TIPO'], df_Machine_Learning], axis=1, sort=False)
final_df

,TIPO,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10
0,Problem,0.286256,-0.339174,-0.573575,-0.884887,0.445071,-0.710361,-1.126215,0.090227,0.164009,0.232113
1,Effect,0.226526,-0.461072,-0.451982,-1.173649,0.149041,-0.719426,-1.211360,0.219315,0.295546,0.043293
2,Effect,0.286788,-0.099284,-0.597585,-1.023533,0.309342,-0.406159,-1.219498,-0.009136,0.246524,0.093233
3,Effect,-0.015221,0.071137,-0.636611,-0.823496,0.532679,-0.486594,-1.117651,-0.290573,0.156254,0.094861
4,Effect,0.198122,-0.386687,-0.580934,-0.974383,0.046443,-0.629261,-1.412848,0.126236,0.013052,0.234437
...,...,...,...,...,...,...,...,...,...,...,...
13029,Cause,-0.243415,-0.028544,-0.437907,-0.704779,0.708401,-0.924312,-1.238950,0.420790,0.304674,-0.039967
13030,Cause,0.137070,-0.323087,-0.522743,-0.786994,0.663143,-0.420877,-1.190545,0.149719,0.301864,0.074213
13031,Cause,0.254662,-0.481095,-0.389147,-1.080241,0.346298,-0.529304,-1.427289,0.028929,0.350832,0.084994
13032,Cause,-0.403198,-0.331759,-0.369534,-0.631055,0.313561,-0.232321,-2.090867,-0.432627,-0.280599,0.698789


In [ ]:
#final_df.to_csv("input_omar_sg.csv",sep=";")

In [ ]:
X=final_df.drop("TIPO",axis=1)

In [ ]:
y=final_df['TIPO']
y

0        Problem
1         Effect
2         Effect
3         Effect
4         Effect
          ...   
13029      Cause
13030      Cause
13031      Cause
13032      Cause
13033      Cause
Name: TIPO, Length: 13034, dtype: object

In [ ]:
y = pd.get_dummies(y, prefix='TIPO')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
def make_model():
    LSTM_1= tf.keras.layers.LSTM(units=3,input_shape=(10,1))
    capa_oculta1= tf.keras.layers.Dense(units=12, activation='sigmoid')
    capa_salida= tf.keras.layers.Dense(units=3,activation='softmax')
    
    modelo= tf.keras.Sequential([LSTM_1,capa_oculta1,capa_salida])
    
    modelo.compile(
        optimizer=keras.optimizers.Adam(0.00005),
        #optimizer=keras.optimizers.RMSprop(0.01),        
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return modelo

In [ ]:
model=make_model()
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_10 (LSTM)              (None, 3)                 60        
                                                                 
 dense_22 (Dense)            (None, 12)                48        
                                                                 
 dense_23 (Dense)            (None, 3)                 39        
                                                                 
Total params: 147
Trainable params: 147
Non-trainable params: 0
_________________________________________________________________


In [ ]:
train_log= model.fit(X_train, y_train,
                    epochs=300 , batch_size=16, validation_data=(X_test,y_test))

Epoch 1/300
611/611 [==============================] - 40s 6ms/step - loss: 1.1482 - accuracy: 0.3602 - val_loss: 1.0987 - val_accuracy: 0.4682
Epoch 2/300
611/611 [==============================] - 3s 5ms/step - loss: 1.0749 - accuracy: 0.4552 - val_loss: 1.0441 - val_accuracy: 0.4682
Epoch 3/300
611/611 [==============================] - 3s 5ms/step - loss: 1.0307 - accuracy: 0.4552 - val_loss: 1.0115 - val_accuracy: 0.4682
Epoch 4/300
611/611 [==============================] - 3s 5ms/step - loss: 1.0036 - accuracy: 0.4552 - val_loss: 0.9915 - val_accuracy: 0.4682
Epoch 5/300
611/611 [==============================] - 3s 5ms/step - loss: 0.9868 - accuracy: 0.4552 - val_loss: 0.9792 - val_accuracy: 0.4679
Epoch 6/300
611/611 [==============================] - 3s 5ms/step - loss: 0.9760 - accuracy: 0.4914 - val_loss: 0.9710 - val_accuracy: 0.5164
Epoch 7/300
611/611 [==============================] - 3s 5ms/step - loss: 0.9684 - accuracy: 0.5097 - val_loss: 0.9645 - val_accuracy: 0.523

In [ ]:
test_predictions = model.predict(X_test)

102/102 [==============================] - 0s 2ms/step


In [ ]:
test_predictions

array([[0.54186773, 0.38391566, 0.07421654],
       [0.7430712 , 0.22222807, 0.03470073],
       [0.87777174, 0.09063061, 0.03159766],
       ...,
       [0.12976673, 0.85725564, 0.01297763],
       [0.15122618, 0.5578811 , 0.2908928 ],
       [0.9066407 , 0.05328276, 0.04007648]], dtype=float32)

In [ ]:
test_predictions=pd.DataFrame(test_predictions)

In [ ]:
m = np.zeros_like(test_predictions.values)
m[np.arange(len(test_predictions)), test_predictions.values.argmax(1)] = 1

df1 = pd.DataFrame(m, columns = test_predictions.columns).astype(int)
y_pred=np.argmax(m,axis=1)

In [ ]:
y_test=y_test.to_numpy()

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(np.argmax(y_test, axis = 1), np.argmax(m,axis=1))

array([[ 979,  263,   47],
       [ 283, 1197,   46],
       [  89,   65,  290]])

In [ ]:
y_test=np.argmax(y_test, axis = 1)
y_pred=np.argmax(m,axis=1)

In [ ]:
target_names = ['Causa', 'Efecto', 'Problema']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

       Causa       0.72      0.76      0.74      1289
      Efecto       0.78      0.78      0.78      1526
    Problema       0.76      0.65      0.70       444

    accuracy                           0.76      3259
   macro avg       0.76      0.73      0.74      3259
weighted avg       0.76      0.76      0.76      3259

